# Sample-2 Model Building

In [99]:
import pandas as pd
import numpy as np
from statsmodels.formula.api import ols,glm
from sklearn.linear_model import LinearRegression, LogisticRegression, Ridge, Lasso, ElasticNet
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet, SGDRegressor, SGDClassifier
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder
import statsmodels.api as sm
import category_encoders as ce
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

In [4]:
from random import random,seed
seed(42)
SAMPLE_SIZE=0.028
da=pd.read_csv('train.csv', skiprows=lambda x: random() > SAMPLE_SIZE)

In [5]:
da

,1000009418151094273,0,14102100,1005,0.1,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,2,15706,320,50,1722,0.2,35,-1,79
0,10001966791793526909,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15701,320,50,1722,0,35,-1,79
1,10004181428767727519,0,14102100,1005,1,0c2fe9d6,27e3c518,28905ebd,ecad2386,7801e8d9,...,1,0,6558,320,50,571,2,39,-1,32
2,10015745448500295401,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15701,320,50,1722,0,35,-1,79
3,10017994765842027302,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,15707,320,50,1722,0,35,-1,79
4,10018099580463747729,0,14102100,1005,0,d6137915,bb1ef334,f028772b,ecad2386,7801e8d9,...,1,0,16920,320,50,1899,0,431,100081,117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1130365,997225830558183390,0,14103023,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,7801e8d9,...,1,0,22676,320,50,2616,0,35,100083,51
1130366,9976695138486904592,0,14103023,1005,1,85f751fd,c4e18dd6,50e219e0,87c134b9,7801e8d9,...,1,0,20508,320,50,2351,3,163,-1,61
1130367,9978195739843146297,0,14103023,1005,0,85f751fd,c4e18dd6,50e219e0,7e7baafa,2347f47a,...,1,0,23866,320,50,2736,0,33,100170,246
1130368,9983622756355579556,1,14103023,1005,1,e151e245,7e091613,f028772b,ecad2386,7801e8d9,...,1,0,22815,320,50,2647,2,39,100148,23


In [6]:
da.columns

Index(['1000009418151094273', '0', '14102100', '1005', '0.1', '1fbe01fe',
       'f3845767', '28905ebd', 'ecad2386', '7801e8d9', '07d7df22', 'a99f214a',
       'ddd2926e', '44956a24', '1', '2', '15706', '320', '50', '1722', '0.2',
       '35', '-1', '79'],
      dtype='object')

In [7]:
da.columns = ['id', 'click', 'hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']

In [8]:
da.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1130370 entries, 0 to 1130369
Data columns (total 24 columns):
 #   Column            Non-Null Count    Dtype 
---  ------            --------------    ----- 
 0   id                1130370 non-null  uint64
 1   click             1130370 non-null  int64 
 2   hour              1130370 non-null  int64 
 3   C1                1130370 non-null  int64 
 4   banner_pos        1130370 non-null  int64 
 5   site_id           1130370 non-null  object
 6   site_domain       1130370 non-null  object
 7   site_category     1130370 non-null  object
 8   app_id            1130370 non-null  object
 9   app_domain        1130370 non-null  object
 10  app_category      1130370 non-null  object
 11  device_id         1130370 non-null  object
 12  device_ip         1130370 non-null  object
 13  device_model      1130370 non-null  object
 14  device_type       1130370 non-null  int64 
 15  device_conn_type  1130370 non-null  int64 
 16  C14               

In [9]:
X = da.drop(columns = ['click','id'])
y = da['click']

In [10]:
X.columns = ['hour', 'C1', 'banner_pos', 'site_id', 'site_domain',
       'site_category', 'app_id', 'app_domain', 'app_category', 'device_id',
       'device_ip', 'device_model', 'device_type', 'device_conn_type', 'C14',
       'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21']
y.columns = ['click']

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=2022)

In [12]:
num_vars = X.select_dtypes(exclude='object').columns
cat_vars = X.select_dtypes(include='object').columns

In [13]:
pipeline = Pipeline([('si',SimpleImputer(strategy='median')),('std',StandardScaler())])

In [14]:
encoder = ce.LeaveOneOutEncoder(return_df=True)
encoder.fit_transform(X_train[cat_vars], y_train)

,site_id,site_domain,site_category,app_id,app_domain,app_category,device_id,device_ip,device_model
74545,0.119250,0.123157,0.128854,0.186599,0.180652,0.093352,0.000000,0.169714,0.264895
375783,0.059667,0.093654,0.281317,0.198247,0.194532,0.198831,0.174038,0.169714,0.182077
258793,0.119250,0.123157,0.128854,0.186599,0.180652,0.093352,0.169714,0.400000,0.160920
894156,0.119250,0.123157,0.128854,0.191496,0.191451,0.108278,0.174038,0.169714,0.169838
488569,0.119250,0.123157,0.128854,0.136242,0.138858,0.090358,0.174038,0.000000,0.225448
...,...,...,...,...,...,...,...,...,...
439985,0.100799,0.259625,0.179209,0.198246,0.194530,0.198829,0.174036,0.097134,0.127920
103024,0.119250,0.123157,0.128854,0.022881,0.123753,0.093352,0.174038,0.083333,0.080258
147629,0.060743,0.060743,0.179213,0.198247,0.194532,0.198831,0.174038,0.000000,0.152768
787676,0.119250,0.123157,0.128854,0.048090,0.048084,0.108278,0.174038,0.169714,0.107210


In [15]:
preprocessing_pipeline = ColumnTransformer([('num',pipeline,num_vars),('cat',encoder,cat_vars)])

In [16]:
X_train_std=pd.DataFrame(preprocessing_pipeline.fit_transform(X_train,y_train))

In [17]:
X_train_std.shape

(904296, 22)

In [18]:
X_test_std=pd.DataFrame(preprocessing_pipeline.fit_transform(X_test,y_test))

In [19]:
X_test_std.shape

(226074, 22)

# Logistic Regression

In [ ]:
lr = LogisticRegression(random_state=2022)

In [ ]:
penalty = ['l1', 'l2','elasticnet']
solver = ['newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga']

In [ ]:
parameter_lr = dict(penalty=penalty, solver=solver)

In [ ]:
lr_random = RandomizedSearchCV( estimator = lr, param_distributions = parameter_lr, n_iter=20, n_jobs=-1,random_state=2022)

In [ ]:
lr_random.fit(X_train_std, y_train)

C:\Users\prabhakaran.p\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:292: UserWarning: The total space of parameters 15 is smaller than n_iter=20. Running 15 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
C:\Users\prabhakaran.p\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:372: FitFailedWarning: 
40 fits failed out of a total of 75.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
5 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\prabhakaran.p\Anaconda3\lib\site-packages\sklearn\model_selection\_validation.py", line 680, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\prabhakaran.p\Anaconda3\lib\sit

RandomizedSearchCV(estimator=LogisticRegression(random_state=2022), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'penalty': ['l1', 'l2', 'elasticnet'],
                                        'solver': ['newton-cg', 'lbfgs',
                                                   'liblinear', 'sag',
                                                   'saga']},
                   random_state=2022)

In [ ]:
y_pred=lr_random.predict(X_test_std)

In [ ]:
y_pred.shape

(226074,)

In [ ]:
pd.crosstab(y_test,lr_random.predict(X_test_std))

col_0,0,1
click,,
0,184759,2858
1,35444,3013


In [ ]:
from sklearn.metrics import accuracy_score

In [ ]:
accuracy_score(y_test,y_pred)

0.8305775984854517

In [ ]:
recall_score(y_test,y_pred)

0.07834724497490704

In [ ]:
precision_score(y_test,y_pred)

0.5132004769204564

# Gaussian Naive bayes classifier

In [82]:
from sklearn.inspection import permutation_importance

In [83]:
from sklearn.model_selection import RepeatedStratifiedKFold

In [84]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

In [85]:
fm_gn=gnb.fit(X_train_std, y_train)

In [86]:
y_predi=gnb.predict(X_test_std)

In [87]:
pd.crosstab(y_test,y_predi)

col_0,0,1
click,,
0,155872,31745
1,23789,14668


In [88]:
imps = permutation_importance(fm_gn, X_test_std, y_test)
imps.importances_mean

array([-0.00022647, -0.00150305, -0.00147208, -0.00121288, -0.00688182,
        0.00057769,  0.00031052,  0.01510921, -0.00038571, -0.00143847,
       -0.00323611, -0.00060069, -0.00269204,  0.0069119 ,  0.00685793,
       -0.00329538, -0.00839371, -0.01260649, -0.00970302,  0.00337854,
        0.00101737, -0.00539204])

In [89]:
accuracy_score(y_test,y_predi)

0.7543547687925193

In [90]:
recall_score(y_test,y_predi)

0.3814130067347947

In [91]:
precision_score(y_test,y_predi)

0.3160321461659449

In [92]:
print(metrics.classification_report(y_test,gnb.predict(X_test_std)))

              precision    recall  f1-score   support

           0       0.87      0.83      0.85    187617
           1       0.32      0.38      0.35     38457

    accuracy                           0.75    226074
   macro avg       0.59      0.61      0.60    226074
weighted avg       0.77      0.75      0.76    226074

